In [1]:
from swda import Transcript
import glob, os
import numpy as np
from collections import namedtuple

DATA_FOLDER = '/n/sd7/trung/csp/data/swb'
from pydub import AudioSegment

In [2]:
print(len(list(glob.glob(os.path.join(DATA_FOLDER, "swda", '**/*.csv')))))

1155


In [3]:
vocab = {}
dlgs = {}
for file in glob.glob(os.path.join(DATA_FOLDER, "swda", '**/*.csv')):
    trans = Transcript(file, os.path.join(DATA_FOLDER, "swda", 'swda-metadata.csv'))
    dlgid = os.path.basename(file)
    dlgs[dlgid] = list(trans.utterances)

In [4]:
print(len(dlgs))

1155


In [ ]:
words = open(os.path.join(DATA_FOLDER, "words.txt")).read().split('\n')
words = {word.split(' ')[0]: word.split(' ')[1] for word in words}
fo = open(os.path.join(DATA_FOLDER, 'inputs.txt'), 'w')
for dlgid in dlgs:
    for utt in dlgs[dlgid]:
        fo.write('%s %d %s\n' %
            (dlgid, 
            utt.utterance_index, 
            ' '.join(['2'] + [words[word.lower()] if word.lower() in words else '0' for word in utt.pos_words()] + ['1'])))
    break

In [6]:
trans_utts = {}
da_utts = {}

Utt = namedtuple("Utt", "id, caller, start, end, act_tag, words, trans_words, npy")

for dlgid in dlgs:
    longid = dlgid.split('.')[0].split('_')[2]
    trans_utts[longid] = {}
    da_utts[longid] = {}
    dlg = dlgs[dlgid]
    
    for c in ['A', 'B']:
        if not os.path.exists(os.path.join(DATA_FOLDER, "wav", longid)):
            os.mkdir(os.path.join(DATA_FOLDER, "wav", longid))
        transfile = os.path.join(DATA_FOLDER, "transcript_penn/alignments/sw%s%s-ms98-a-penn.text" % (longid, c))
        if not os.path.exists(transfile): 
            print("(not existed: %s%s)" % (longid, c), end=' ')
            continue
            
        with open(transfile) as f:
            lines = f.read().split('\n')
            lines = [line.split('\t') for line in lines]
            lines = [(float(line[2]), float(line[3]), int(line[1].split('.')[-1]), line[5]) for line in lines if len(line) == 7]
            
            cur = None
            for i in range(len(lines)):
                line = lines[i]
                if line[3] in ['[silence]', '[noise]', '[laughter]', '[vocalized-noise]', '---']: continue
                if cur is not None and line[2] == cur[2]:
                    cur = (cur[0], line[1], line[2], cur[3] + [line[3]])
                else:
                    if cur is not None:
                        trans_utts[longid][id] = cur
                    cur = (line[0], line[1], line[2], [line[3]])
                    id = line[2]
            trans_utts[longid][id] = cur # (start, end, id, text)

(not existed: 3942A) (not existed: 3942B) (not existed: 2752A) (not existed: 2752B) (not existed: 2753A) (not existed: 2753B) (not existed: 3994A) (not existed: 3994B) (not existed: 4643A) (not existed: 4643B) (not existed: 4384A) (not existed: 4384B) (not existed: 3968A) (not existed: 3968B) (not existed: 3035A) (not existed: 3035B) (not existed: 2335A) (not existed: 2335B) (not existed: 3520A) (not existed: 3520B) (not existed: 4102A) (not existed: 4102B) (not existed: 2347A) (not existed: 2347B) (not existed: 2733A) (not existed: 2733B) (not existed: 4771A) (not existed: 4771B) (not existed: 2838A) (not existed: 2838B) (not existed: 3936A) (not existed: 3936B) (not existed: 2505A) (not existed: 2505B) (not existed: 4157A) (not existed: 4157B) (not existed: 2632A) (not existed: 2632B) (not existed: 4002A) (not existed: 4002B) (not existed: 3924A) (not existed: 3924B) (not existed: 2702A) (not existed: 2702B) (not existed: 3528A) (not existed: 3528B) (not existed: 4167A) (not existed:

In [17]:
dlg_utts = {}

for dlgid in dlgs:
    longid = dlgid.split('.')[0].split('_')[2]
    dlg = dlgs[dlgid]
    dlg_utts[longid] = []
    if longid not in trans_utts: continue
    cur_id = -1
    for i in range(len(dlg)):
        utt = dlg[i]
        if utt.utterance_index == cur_id:
            cur += utt.pos_words()
        else:
            if cur_id != -1:
                da_utts[longid][cur_id] = (cur_id, c, cur)
                if cur_id in trans_utts[longid] and trans_utts[longid][cur_id][1] - trans_utts[longid][cur_id][0] > 0:
                    dlg_utts[longid].append(Utt(
                        id=cur_id,
                        caller=caller,
                        start=trans_utts[longid][cur_id][0], 
                        end=trans_utts[longid][cur_id][1],
                        act_tag=act_tag,
                        words=cur,
                        trans_words=trans_utts[longid][cur_id][3],
                        npy=os.path.join(DATA_FOLDER, "features", "npy", longid, "%s_%s.npy" % (cur_id, caller))
                    ))
            cur = utt.pos_words()
            cur_id = utt.utterance_index
            caller = utt.caller
            act_tag = utt.act_tag
        #print(da_utts[longid].keys())
    da_utts[longid][cur_id] = cur

    dlg_utts[longid].sort(key=lambda utt: utt.id)

In [35]:
for utt in dlg_utts['3001']:
    print(utt.caller, utt.start, utt.end)
    print(utt.words)
    print(utt.trans_words)
    print()

A 0.461375 0.94525
['Okay', '.']
['Okay']

B 1.64325 2.473625
['Are', 'you', 'a', 'T', 'I', '-er', '?']
['Are', 'you', 'a', 'T', 'I', '-er']

A 3.4235 3.821375
['No', '.']
['No']

B 4.26025 6.95475
['No', '.', 'Is', 'somebody', 'in', 'your', 'family', 'a', 'T', 'I', '-er', '?']
['No', 'Is', 'somebody', 'in', 'your', 'family', 'a', 'T', 'I', '-er']

A 4.850875 12.98425
['No', ',', 'and', 'that', "'s", 'how', 'he', 'got', 'it', '.']
['No', 'uh', 'my', 'fianc3e', 'works', 'for', 'the', 'computers', 'for', 'the', 'government', 'and', "that's", 'how', 'he', 'got', 'it']

B 13.829625 16.376375
['Oh', ',', 'okay', ',', 'so', 'who', 'do', 'you', 'work', 'for', '?']
['Oh', 'okay', 'so', 'who', 'do', 'you', 'work', 'for']

A 17.044875 18.834875
['I', 'just', 'graduated', 'from', 'college', '.']
['I', 'just', 'graduated', 'from', 'college']

B 19.09 21.22
['So', 'you', 'do', "n't", 'have', 'any', 'benefits', 'at', 'all', 'to', 'talk', 'about', '.']
['So', 'you', "don't", 'have', 'any', 'benefits'

In [20]:
dlg_utts = { id: dlg_utts[id] for id in dlg_utts if len(dlg_utts[id]) > 0 }
print("Conversations:", len(dlg_utts))
print("Utterances:", sum([len(dlg_utts[id]) for id in dlg_utts]))
print("Utterances' Length: %.2f hours" % (sum([sum([utt.end - utt.start for utt in dlg_utts[id]]) for id in dlg_utts]) / 3600))
print("Length: %.2f hours" % (sum([dlg_utts[id][-1].end for id in dlg_utts]) / 3600))

Conversations: 1126
Utterances: 117123
Utterances' Length: 119.60 hours
Length: 121.10 hours


In [41]:
vocab = {}
mode = "train"
dlg_keys = list(dlg_utts.keys())[:10] if mode == "test" else list(dlg_utts.keys())[10:]
for longid in dlg_keys:
    utts = dlg_utts[longid]
    for utt in utts:
        for word in utt.trans_words:
            word = word.lower()
            if word in vocab: vocab[word] += 1
            else: vocab[word] = 1
                
# print(len([word for word in vocab if vocab[word] == 1]))

vocab2 = [word for word in vocab if vocab[word] > 1]
print(len(vocab2))
vocab2.sort()
with open(os.path.join(DATA_FOLDER, "words.txt"), 'w') as f:
    f.write('<unk> 0\n<sos> 2\n<eos> 1\n')
    f.write('\n'.join(['%s %d' % (word, i + 3) for i, word in enumerate(vocab2)]))

13242


In [39]:
mode = "train"
dlg_keys = list(dlg_utts.keys())[:10] if mode == "test" else list(dlg_utts.keys())[10:]

In [40]:
from subprocess import call
from struct import unpack, pack

mean = np.load(os.path.join(DATA_FOLDER, "mean.npy"))
var = np.load(os.path.join(DATA_FOLDER, "var.npy"))

outputs = []
#mean = np.array([0] * 120)
#var = np.array([0] * 120)
count = 0

for longid in dlg_keys:
    print(longid, end=' ')
    wav = {}
    for c in ['A', 'B']:
        wavpath = os.path.join(DATA_FOLDER, "wav/sw0%s_%s.wav" % (longid, c))
        wav[c] = AudioSegment.from_wav(wavpath)
    utts = dlg_utts[longid]
    
    if not os.path.exists(os.path.join(DATA_FOLDER, "features", "wav", longid)):
        os.mkdir(os.path.join(DATA_FOLDER, "features", "wav", longid))
    if not os.path.exists(os.path.join(DATA_FOLDER, "features", "npy", longid)):
        os.mkdir(os.path.join(DATA_FOLDER, "features", "npy", longid))
    if not os.path.exists(os.path.join(DATA_FOLDER, "features", "htk", longid)):
        os.mkdir(os.path.join(DATA_FOLDER, "features", "htk", longid))
    # count += 1
    
    for utt in utts:
        id = utt.id
        c = utt.caller
        output_wav = os.path.join(DATA_FOLDER, "features", "wav", longid, "%s_%s.wav" % (id, c))
        output_npy = os.path.join(DATA_FOLDER, "features", "npy", longid, "%s_%s.npy" % (id, c))
        output_htk = os.path.join(DATA_FOLDER, "features", "htk", longid, "%s_%s.htk" % (id, c))
            
        if True: #
            utt_wav = AudioSegment.silent(500) + wav[c][int(utt.start * 1000):int(utt.end * 1000)] + AudioSegment.silent(500)
            utt_wav = wav[c][int(utt.start * 1000):int(utt.end * 1000)]
            utt_wav.export(output_wav, format='wav', bitrate=16000)

            call([
                "/n/sd7/trung/bin/htk/HTKTools/HCopy",
                output_wav,
                output_htk,
                "-C", "/n/sd7/trung/config.lmfb.40ch"
            ])
        
        fh = open(output_htk, "rb")
        spam = fh.read(12)
        nSamples, sampPeriod, sampSize, parmKind = unpack(">IIHH", spam)
        veclen = int(sampSize / 4)
        fh.seek(12, 0)
            
        dat = np.fromfile(fh, dtype=np.float32)
        dat = dat.reshape(len(dat) // veclen, veclen)
        dat = dat.byteswap()
        
        #for k in range(len(dat)):
        #    updated_mean = (mean * count + dat[k]) / (count + 1)
        #    var = (count * var + (dat[k] - mean) * (dat[k] - updated_mean)) / (count + 1)
        #    mean = updated_mean
        #    count += 1
        
        dat = (dat - mean) / np.sqrt(var)
        print(output_wav, len(dat))
                
        fh.close()
        np.save(output_npy, dat)
    
#np.save(os.path.join(DATA_FOLDER, "mean.npy"), mean)
#np.save(os.path.join(DATA_FOLDER, "var.npy"), var)

4078 /n/sd7/trung/csp/data/swb/features/wav/4078/1_A.wav 291
/n/sd7/trung/csp/data/swb/features/wav/4078/2_B.wav 823
/n/sd7/trung/csp/data/swb/features/wav/4078/3_A.wav 28
/n/sd7/trung/csp/data/swb/features/wav/4078/4_B.wav 355


KeyboardInterrupt: 

In [32]:
words = open(os.path.join(DATA_FOLDER, "words.txt")).read().split('\n')
words = {word.split(' ')[0]: word.split(' ')[1] for word in words}
outputs = []
for longid in dlg_keys:
    utts = dlg_utts[longid]
    for utt in utts:
        if len(utt.trans_words) > 0:
            outputs.append("%s %s" % (
                utt.npy,
                ' '.join(['2'] + [words[w.lower()] if w.lower() in words else '0' for w in utt.trans_words] + ['1'])
            ))
        
outputs.sort(key=lambda o: len(o))
with open(os.path.join(DATA_FOLDER, 'inputs_%s.txt' % (mode)), 'w') as fo:
    fo.write('\n'.join(outputs))